## Task 1: Sentiment analysis of twitter and stockwits data

In [1]:
import json
from pprint import pprint
import matplotlib as mpl
import matplotlib.pyplot as pltfrom 

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np
import pandas as pd

from keras.layers import Flatten
from keras import regularizers
from sklearn.metrics.pairwise import cosine_similarity

Using TensorFlow backend.


### Loading data from JSON

In [2]:
with open('Microblog_Trainingdata.json') as data_file:    
    Microblog_Trainingdata = json.load(data_file)

for i in range(0,len(Microblog_Trainingdata)):
    Microblog_Trainingdata[i]['spans']=Microblog_Trainingdata[i]['spans'][0]
    Microblog_Trainingdata[i]['sentiment score']=float(Microblog_Trainingdata[i]['sentiment score'])
#pprint(Microblog_Trainingdata)
with open('Microblog_Trialdata.json') as data_file:   
    Microblog_Trialdata = json.load(data_file)
    

#pprint(data)
for i in range(0,len(Microblog_Trialdata)):
    Microblog_Trialdata[i]['spans']=Microblog_Trialdata[i]['spans'][0]
    Microblog_Trialdata[i]['sentiment score']=float(Microblog_Trialdata[i]['sentiment score'])

In [3]:
with open('Microblogs_Testdata.json') as data_file:   
    Microblogs_Testdata = json.load(data_file)
    
#pprint(Microblog_Testdata)

In [4]:
for i in Microblog_Trialdata:
    Microblog_Trainingdata.append(i)

In [5]:
def lower_keys(x):
    if isinstance(x, list):
        return [lower_keys(v) for v in x]
    elif isinstance(x, dict):
        return dict((k.lower(), v.lower() if isinstance(v, str) else v) for k, v in x.items())
    else:
        return x
Microblog_Trainingdata=lower_keys(Microblog_Trainingdata)

In [6]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [7]:
import string
def stringClean( inputString ) :
    return inputString.translate(str.maketrans('','',string.punctuation)).encode("ascii", errors="ignore").decode()

In [8]:
def removeStopWords(inputString):
    splitString = inputString.split()
    for word in stop:
        for s in splitString:
            if s == word:
                splitString.remove(s)
    return " ".join(splitString)

In [9]:
def calculateCosineSimilarity( pVal,gVal ):
    similarity = cosine_similarity(pVal,gVal)
    pNorm = np.linalg.norm(pVal)
    gNorm = np.linalg.norm(gVal)
    wt = pNorm/gNorm
    return wt * similarity[0][0]

### Loading Testing and Testing data

In [10]:
maxlen = 0

X_train = []
X_trial=[]
Y_trial=[]
X_test = [Microblog['spans'] for Microblog in Microblogs_Testdata]
Y_train = []
Y_test = []
output=[]
for Microblog in Microblog_Trainingdata :
    X_train.append(removeStopWords(stringClean(Microblog['spans'])))
    Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
    output.append(Microblog['sentiment score'])
    if maxlen < len(Microblog['spans'].split()) :
        maxlen = len(Microblog['spans'].split())

#for Microblog in Microblog_Trialdata :
 #   X_train.append(Microblog['spans'])
  #  Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
   # if maxlen < len(Microblog['spans'].split()) :
    #    maxlen = len(Microblog['spans'].split())


print(len(X_train)+len(X_test), len(Y_train))


2510 1710


### Tokenize the data and pad shorter reviews

In [11]:
tokenizer = Tokenizer(num_words=maxlen*20)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
data = pad_sequences(sequences, maxlen)
data.shape

(1710, 25)

### LSTM Model

In [12]:
model = Sequential()
model.add(Embedding(maxlen*20, 32, input_length=maxlen))
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='leakyrelu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.15))
#, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)
model.add(Dense(4, activation='linear'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 32)            16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 24,593
Trainable params: 24,593
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(data, np.array(Y_train), validation_split=0.2, epochs=15,verbose = 1)

Train on 1368 samples, validate on 342 samples
Epoch 1/15
1368/1368 [==============================] - 3s - loss: 1.7079 - acc: 0.3567 - val_loss: 0.8281 - val_acc: 0.3626
Epoch 2/15
1368/1368 [==============================] - 1s - loss: 0.7051 - acc: 0.5490 - val_loss: 0.6491 - val_acc: 0.6491
Epoch 3/15
1368/1368 [==============================] - 1s - loss: 0.6466 - acc: 0.6462 - val_loss: 0.6245 - val_acc: 0.6579
Epoch 4/15
1368/1368 [==============================] - 1s - loss: 0.6255 - acc: 0.6806 - val_loss: 0.5998 - val_acc: 0.6813
Epoch 5/15
1368/1368 [==============================] - 1s - loss: 0.5807 - acc: 0.7288 - val_loss: 0.5615 - val_acc: 0.7368
Epoch 6/15
1368/1368 [==============================] - 1s - loss: 0.5192 - acc: 0.7800 - val_loss: 0.5085 - val_acc: 0.7778
Epoch 7/15
1368/1368 [==============================] - 1s - loss: 0.4404 - acc: 0.8209 - val_loss: 0.4616 - val_acc: 0.7749
Epoch 8/15
1368/1368 [==============================] - 1s - loss: 0.3933 - ac

In [14]:
sequences = tokenizer.texts_to_sequences(np.array(X_train))
data = pad_sequences(sequences, maxlen)
predictions = model.predict(data)
for i in range(len(predictions)):
    print(Microblog_Trainingdata[i]['spans'],Microblog_Trainingdata[i]['sentiment score'],"%.3f" % predictions[i])

watching for bounce tomorrow 0.366 0.899
record number of passengers served in 2015 0.638 0.998
out $nflx -.35 -0.494 -0.002
looking for a strong bounce 0.46 0.986
very intrigued with the technology and growth potential 0.403 0.957
short worked 0.0 -0.131
overbought -0.296 -0.295
absolute garbage still up -0.546 0.651
biggest market losers -0.438 -0.379
love this company long time. 0.408 0.829
$goog $googl would suck -0.398 0.075
who won't pay anymore -0.349 -0.457
no edge offered 0.025 0.984
runs into the 50sma on the acquisition news 0.486 0.735
t can't go down 0.308 0.875
now seems like its helping the downtrend -0.372 -0.193
mastered their supply chain 0.461 0.695
most bullish stocks on twitter during this dip 0.408 0.992
big dumping -0.699 0.766
trade continuing very nicely from yesterday 0.495 0.997
in from 3.61 0.306 0.695
stochastic overbought -0.385 -0.620
unusual call buying 0.336 0.989
reserves are in decline 0.279 0.695
all on a longer-term, technical swing long basis. 0.59

biggest market losers -0.438 -0.379
with personal story sharing declining -0.393 -0.513
upgrades 0.334 0.943
saying this is a peak = fear mongering 0.329 0.695
the most trending 0.216 0.958
who won't pay anymore -0.349 -0.457
still they are saying bullish lol -0.038 0.900
today's bull flag 0.2 0.934
growth rate of approximately 122.0% to 124.9%. 0.305 0.823
lots of nice setups in energy  0.293 0.958
bad governance -0.331 0.214
airplane and hospitality industries set their sights on #cuba 0.405 0.999
landed in $ea 0.422 0.695
bought $skf as a hedge 0.466 0.984
transports always lead the way 0.594 0.988
here is no case here 0.18 0.695
$joe getting attention. no.2 on nyse 100 top buying list 0.788 0.967
short setups looking nice....really nice -0.464 -0.076
highest close since 0.387 0.675
should bring $spx down into the close. -0.462 0.615
 0.295 0.695
money flow & stochastic dropping -0.577 -0.432
9% run for this watch list winner 0.066 0.999
have had 2 wks of fund inflows for the first 

ready for take off! 0.472 0.634
report: apple signs up for google's cloud 0.15 0.423
short setups looking nice....really nice -0.464 -0.076
short $mbly -0.606 -0.131
will push $nly investment to another time -0.255 -0.296
notable gainers among liquid option names this morning 0.513 0.994
looking to add a gold miner and possibly lower pe tech name into close 0.425 0.936
point at a possible move up 0.176 0.960
they let the stock fall big time after earnings -0.455 -0.110
not selling before 0.221 0.519
tremendous opportunity awaits 0.456 0.930
up 10% in premarket 0.507 0.716
the overall market improve 0.433 0.615
gots some $aezs this morning 0.514 0.873
erinn murphy of piper jaffray thinks $finl is a buy 0.501 0.891
longer term top building 0.099 0.803
debt crisis entered a dangerous new phase -0.331 -0.424
$mu may go down 6% if the bear pattern is confirmed -0.404 -0.291
looking good 0.255 0.772
cracking -0.118 0.695
bullish ma crossovers 0.457 0.981
still an interesting finance play 0.3

downbeaten stocks may be ripe for buys 0.316 0.994
buy safety equipment maker for $2.5 billion 0.23 0.779
weak close -0.297 -0.264
looks like it could be about to fall apart -0.515 0.003
 0.0 0.695
shorts are getting nervous 0.366 0.198
facebook has now been down 10 minutes -0.258 0.948
monday bulls are out! 0.457 0.910
today's top gainers 0.447 0.990
tesla motors recalls -0.424 0.438
aggressive buy 0.306 0.975
mentioned as potential suitee, suitor 0.258 0.930
we see more reasons to be cautious -0.093 -0.275
do you really think in 4 weeks they capture 10%? impossible. -0.317 -0.055
friday was last chance under $2 0.228 0.779
excellent / factual bear argument -0.507 -0.406
bears take a clear near term advantage -0.501 0.187
relative weakness is pretty apparent -0.262 0.677
don't think buyout rumor strong enough too be driving this up just yet -0.02 0.830
$yhoo +2.61% 0.16 0.849
$tplm triangle #petroleum 9.80+3% 2m expanding caliber 0.376 0.943
downbeaten stocks may be ripe for buys 0.31

### Computing Cosine score

In [15]:
#predictedVals = [pr[0] for pr in predictions ]

In [16]:
predictedVals = [pr[0] for pr in predictions ]
output = np.array(output).reshape(1,-1)
predictedVals = np.array(predictedVals).reshape(1,-1)


print("Final cosine score :",cosine_similarity(predictedVals,output)[0][0])

Final cosine score : 0.638443989711


In [17]:
df=pd.DataFrame.from_dict(Microblog_Trainingdata)

In [18]:
df['Predicted Sentiment Score']=predictions
df.head()

,cashtag,id,sentiment score,source,spans,Predicted Sentiment Score
0,$fb,719659409228451840,0.366,twitter,watching for bounce tomorrow,0.898739
1,$luv,719904304207962112,0.638,twitter,record number of passengers served in 2015,0.998171
2,$nflx,5329774,-0.494,stocktwits,out $nflx -.35,-0.001671
3,$dia,719891468173844480,0.460,twitter,looking for a strong bounce,0.985798
4,$plug,20091246,0.403,stocktwits,very intrigued with the technology and growth ...,0.957454


In [19]:
task1Dataframe=pd.DataFrame({"ID":df['id'],"Spans":df['spans'],"Cashtag":df['cashtag'],"Sentiment Score":df['sentiment score'],"Predicted Sentiment Score":df['Predicted Sentiment Score']})
task1Dataframe=task1Dataframe[['ID','Spans','Cashtag','Sentiment Score','Predicted Sentiment Score']]
task1Dataframe

,ID,Spans,Cashtag,Sentiment Score,Predicted Sentiment Score
0,719659409228451840,watching for bounce tomorrow,$fb,0.366,0.898739
1,719904304207962112,record number of passengers served in 2015,$luv,0.638,0.998171
2,5329774,out $nflx -.35,$nflx,-0.494,-0.001671
3,719891468173844480,looking for a strong bounce,$dia,0.460,0.985798
4,20091246,very intrigued with the technology and growth ...,$plug,0.403,0.957454
5,5819749,short worked,$gmcr,0.000,-0.131187
6,709741154393133056,overbought,$ibm,-0.296,-0.295217
7,17892972,absolute garbage still up,$josb,-0.546,0.651011
8,709834259687710720,biggest market losers,$cstm,-0.438,-0.378919
9,708481442079068160,love this company long time.,$pypl,0.408,0.829186


In [20]:
#Convert dataframe to csv file
task1Dataframe.to_csv('Task1.csv',index=False)